### Selenium de valores en descenso

En este notebook se ha creado un programa el cual:
* Entra en la información de todos los jugadores
* Extrae su información histórica (Valor de mercado, equipo, edad, fecha)

In [1]:
#Servicio y driver de Chrome de Selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

#Las opciones que vamos a tener para buscar elementos
from selenium.webdriver.common.by import By

#Para cuando queramos mandar pulsaciones de teclado
from selenium.webdriver.common.keys import Keys

from selenium.webdriver.common.action_chains import ActionChains

from bs4 import BeautifulSoup as bs
import requests
import datetime as dt
import time
import numpy as np
import pandas as pd
import openpyxl

Abrimos el driver

In [2]:
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)
time.sleep(5)

[WDM] - Downloading: 100%|██████████| 6.80M/6.80M [00:00<00:00, 10.4MB/s]


Abrimos los links que necesitamos y limpiamos el DataFrame, quitando los "No veo la página" que obtuvimos anteriormente

In [7]:
df_links_mal = pd.read_excel('../data/LinksRegu_Bajadas.xlsx', index_col=0)

,Links
0,No veo la página
1,No veo la página
2,No veo la página
3,No veo la página
4,No veo la página
...,...
427631,No veo la página
427632,No veo la página
427633,No veo la página
427634,No veo la página


Aquí quitamos los "No veo la página"

In [4]:
df_links_normal = df_links_mal[df_links_mal['Links'] != 'No veo la página'].reset_index(drop=True)
df_links_normal.iloc[0,0]

'https://www.transfermarkt.es/romelu-lukaku/marktwertverlauf/spieler/96341'

#### Empezamos con el programa

Primero, tenemos que aceptar las cookies (y tratamos de quitar el primer pop up)

En ocasiones, al pulsar sobre la x del pop-up es posible que se abra una ventana nueva. El programa está preparado para cerrarla en caso de que eso suceda :)

In [41]:
lista_datos = []


driver.get(df_links_normal.iloc[0,0])
time.sleep(1.5)

try:
    iframe = driver.find_elements(By.TAG_NAME,'iframe')[-1]
    driver.switch_to.frame(iframe)
    time.sleep(1)

    aceptar = driver.find_elements(By.TAG_NAME, 'button')[-1]
    aceptar.click()
    
except:
    print('Na')
time.sleep(5)

try:
    cerrar_pop_up = driver.find_elements(By.TAG_NAME, 'span')

    for x in cerrar_pop_up:
        if "×" in x.text:
            x.click()
            time.sleep(1)
            try:
                iframe = driver.find_elements(By.TAG_NAME,'iframe')[-1]
                driver.switch_to.frame(iframe)
                time.sleep(1)

                aceptar = driver.find_elements(By.TAG_NAME, 'button')[-1]
                aceptar.click()
                
            except:
                print('Na')
        else:
            continue
except:
    print('na')

try:
    if len(set(driver.window_handles))>1:
        driver.switch_to.window(driver.window_handles[1])
        driver.close()
    driver.switch_to.window(driver.window_handles[0])
except:
    print('NA')

### El Programa se ejecuta aquí

In [42]:
lista_datos = []

for url in range(len(df_links_normal['Links'])):
    try:
        driver.get(df_links_normal.iloc[url,0])
        time.sleep(1.7)
        
        try:
            cerrar_pop_up = driver.find_elements(By.TAG_NAME, 'span')

            for x in cerrar_pop_up:
                if "×" in x.text:
                    x.click()
                    time.sleep(1)
                else:
                    continue
            
        except:
            print('Na')
        try:
            if len(set(driver.window_handles))>1:
                driver.switch_to.window(driver.window_handles[1])
                driver.close()
            driver.switch_to.window(driver.window_handles[0])
        except:
            continue
        time.sleep(0.5)

        logos = driver.find_elements(By.TAG_NAME, 'image') #Primera parte
        #Aquí irán los datos
        nombre = driver.find_element(By.CLASS_NAME, 'data-header__headline-wrapper').text

        if logos != '':
        #bucle de la primera parte
            for i in logos: 
            #Action Chains sirve para crear cadenas de acción. Es algo más avanzado pero no es muy complicado
                ActionChains(driver).move_to_element(i).perform() #Movemos el cursor hacia el elemento
    
                #hacemos que descanse dos segundos
    
            #Quiero todos los elementos (find_elements)
                Elementos=driver.find_elements(By.CLASS_NAME, 'highcharts-tooltip') 

            #bucle en elementos
                for j in Elementos:
                    time.sleep(0.2)
                
                    print(j.text)

                #lo añado a la lista separando los valores por saltos de línes
                    lista_datos.append(j.text.split('\n') + [nombre])
    except:
        url-=1
        continue


30/03/2023
Valor de mercado: 450 mil €
Club: Śląsk Wroclaw
Edad : 31

16/11/2022
Valor de mercado: 550 mil €
Club: Śląsk Wroclaw
Edad : 31

29/09/2022
Valor de mercado: 450 mil €
Club: Śląsk Wroclaw
Edad : 30

14/06/2022
Valor de mercado: 600 mil €
Club: Śląsk Wroclaw
Edad : 30

07/04/2022
Valor de mercado: 800 mil €
Club: Śląsk Wroclaw
Edad : 30

31/12/2021
Valor de mercado: 1,00 mill. €
Club: Śląsk Wroclaw
Edad : 30

15/10/2021
Valor de mercado: 1,20 mill. €
Club: Śląsk Wroclaw
Edad : 29

21/05/2021
Valor de mercado: 1,50 mill. €
Club: Rakow Czestochowa
Edad : 29

19/01/2021
Valor de mercado: 900 mil €
Club: Rakow Czestochowa
Edad : 29

18/08/2020
Valor de mercado: 600 mil €
Club: Rakow Czestochowa
Edad : 28

08/04/2020
Valor de mercado: 300 mil €
Club: Rakow Czestochowa
Edad : 28

15/01/2020
Valor de mercado: 350 mil €
Club: Rakow Czestochowa
Edad : 28

29/06/2019
Valor de mercado: 175 mil €
Club: Rakow Czestochowa
Edad : 27

21/02/2019
Valor de mercado: 75 mil €
Club: Rakow Czesto

Vemos una muestra de como se obtienen los datos

In [43]:
lista_datos

[['', '#17 Petr Schwarz'],
 ['30/03/2023',
  'Valor de mercado: 450 mil €',
  'Club: Śląsk Wroclaw',
  'Edad : 31',
  '#17 Petr Schwarz'],
 ['', '#17 Petr Schwarz'],
 ['16/11/2022',
  'Valor de mercado: 550 mil €',
  'Club: Śląsk Wroclaw',
  'Edad : 31',
  '#17 Petr Schwarz'],
 ['', '#17 Petr Schwarz'],
 ['29/09/2022',
  'Valor de mercado: 450 mil €',
  'Club: Śląsk Wroclaw',
  'Edad : 30',
  '#17 Petr Schwarz'],
 ['', '#17 Petr Schwarz'],
 ['14/06/2022',
  'Valor de mercado: 600 mil €',
  'Club: Śląsk Wroclaw',
  'Edad : 30',
  '#17 Petr Schwarz'],
 ['', '#17 Petr Schwarz'],
 ['07/04/2022',
  'Valor de mercado: 800 mil €',
  'Club: Śląsk Wroclaw',
  'Edad : 30',
  '#17 Petr Schwarz'],
 ['', '#17 Petr Schwarz'],
 ['31/12/2021',
  'Valor de mercado: 1,00 mill. €',
  'Club: Śląsk Wroclaw',
  'Edad : 30',
  '#17 Petr Schwarz'],
 ['', '#17 Petr Schwarz'],
 ['15/10/2021',
  'Valor de mercado: 1,20 mill. €',
  'Club: Śląsk Wroclaw',
  'Edad : 29',
  '#17 Petr Schwarz'],
 ['', '#17 Petr Schwa

##### Limpiamos la lista obtenida

In [44]:
lista_datos_limpia = [] #Nueva lista que NO TENDRÁ listas vacías dentro


for i in lista_datos:

    #Le digo que me añada todo lo que no sean listas vacías en una nueva lista
    if len(i) != 2 :
        lista_datos_limpia.append(i)
print(lista_datos_limpia)

[['30/03/2023', 'Valor de mercado: 450 mil €', 'Club: Śląsk Wroclaw', 'Edad : 31', '#17 Petr Schwarz'], ['16/11/2022', 'Valor de mercado: 550 mil €', 'Club: Śląsk Wroclaw', 'Edad : 31', '#17 Petr Schwarz'], ['29/09/2022', 'Valor de mercado: 450 mil €', 'Club: Śląsk Wroclaw', 'Edad : 30', '#17 Petr Schwarz'], ['14/06/2022', 'Valor de mercado: 600 mil €', 'Club: Śląsk Wroclaw', 'Edad : 30', '#17 Petr Schwarz'], ['07/04/2022', 'Valor de mercado: 800 mil €', 'Club: Śląsk Wroclaw', 'Edad : 30', '#17 Petr Schwarz'], ['31/12/2021', 'Valor de mercado: 1,00 mill. €', 'Club: Śląsk Wroclaw', 'Edad : 30', '#17 Petr Schwarz'], ['15/10/2021', 'Valor de mercado: 1,20 mill. €', 'Club: Śląsk Wroclaw', 'Edad : 29', '#17 Petr Schwarz'], ['21/05/2021', 'Valor de mercado: 1,50 mill. €', 'Club: Rakow Czestochowa', 'Edad : 29', '#17 Petr Schwarz'], ['19/01/2021', 'Valor de mercado: 900 mil €', 'Club: Rakow Czestochowa', 'Edad : 29', '#17 Petr Schwarz'], ['18/08/2020', 'Valor de mercado: 600 mil €', 'Club: Ra

#### Creamos el DataFrame

In [45]:
lista_df = []
for i in range(len(lista_datos_limpia)):
    data = {"Nombre":lista_datos_limpia[i][4],
    "Fecha de actualización": lista_datos_limpia[i][0],
    "Valor de mercado": lista_datos_limpia[i][1],
    "Club": lista_datos_limpia[i][2],
    "Edad": lista_datos_limpia[i][3],

    }
    lista_df.append(data)

In [46]:
df_new = pd.DataFrame(lista_df)
df_new.tail(15)

,Nombre,Fecha de actualización,Valor de mercado,Club,Edad
914,#19 Manuel Insaurralde,26/10/2022,Valor de mercado: 525 mil €,Club: Club de Gimnasia y Esgrima La Plata,Edad : 23
915,#19 Manuel Insaurralde,24/05/2022,Valor de mercado: 550 mil €,Club: Club de Gimnasia y Esgrima La Plata,Edad : 23
916,#19 Manuel Insaurralde,20/10/2021,Valor de mercado: 600 mil €,Club: Club de Gimnasia y Esgrima La Plata,Edad : 22
917,#19 Manuel Insaurralde,30/12/2020,Valor de mercado: 600 mil €,Club: CD Universidad Católica,Edad : 21
918,#19 Manuel Insaurralde,14/10/2020,Valor de mercado: 600 mil €,Club: CD Universidad Católica,Edad : 21
919,#19 Manuel Insaurralde,08/04/2020,"Valor de mercado: 1,10 mill. €",Club: CD Universidad Católica,Edad : 21
920,#19 Manuel Insaurralde,05/12/2019,"Valor de mercado: 1,25 mill. €",Club: CA San Lorenzo de Almagro,Edad : 20
921,#19 Manuel Insaurralde,06/08/2019,"Valor de mercado: 1,50 mill. €",Club: CA San Lorenzo de Almagro,Edad : 20
922,#19 Manuel Insaurralde,18/12/2018,Valor de mercado: 150 mil €,Club: CA San Lorenzo de Almagro,Edad : 19
923,#10 Hernandes,27/12/2022,Valor de mercado: 525 mil €,Club: Incheon United,Edad : 23


Limpiamos cada columna

In [47]:
df_new['Fecha de actualización'] = pd.to_datetime(df_new['Fecha de actualización'], dayfirst=True)

In [48]:
### Limpiamos el DF
### Nombres
nombres_limpio = []
for i in df_new['Nombre']:
    nombres_limpio.append(' '.join(i.split()[1:]))

df_new['Nombre'] = nombres_limpio

In [49]:
clubes_limpio = []
for i in range(len(df_new['Club'])):
    clubes_limpio.append(" ".join(df_new['Club'][i].split()[1:]))
df_new['Club'] = clubes_limpio

In [50]:
edad_limpio = []
for i in df_new['Edad']:
    edad_limpio.append(int(i.split()[-1]))

df_new['Edad']  = edad_limpio

Exportamos a Excel

In [51]:
df_new.to_excel('../data/Selenium_Bajadas.xlsx')